In [49]:
import os
import asyncio

import pandas as pd
import pyodbc

from reclaim_tiktok.scrapping.scrapper import Scrapper

import nest_asyncio
nest_asyncio.apply()

from dotenv import load_dotenv

load_dotenv()
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Get usernames of afd politicans from google sheet of martin fuchs

In [4]:
sheet_df = pd.read_csv('/Users/matthiaskiller/Downloads/Kopie von Deutsche Politik auf Tik Tok - Eine Sammlung von Martin Fuchs @wahl_beobachter - Deutsche Politik auf Tik Tok .csv')

In [5]:
sheet_df.head()

,Unnamed: 0,Name,Tik Tok-Profil,Partei,aktiv,Anmerkungen,In fanpage karma
0,NaN,AfD,https://www.tiktok.com/@afd.offiziell,AfD,NaN,"Aktiv seit 10.05.2021, gelöscht 5/2022 von Tik...",NaN
1,NaN,AfD NRW,https://www.tiktok.com/@afd.nrw,AfD,x,NaN,NaN
2,NaN,AfD Hessen,https://www.tiktok.com/@afdhessen,AfD,NaN,Nicht mehr auffindbar,NaN
3,NaN,AfD Bayern,https://www.tiktok.com/@afd.bayern,AfD,NaN,Alter Account @afd_bayern nicht megr auffindbar,NaN
4,NaN,AfD Brandenburg,https://www.tiktok.com/@afd_brandenburg,AfD,x,NaN,NaN


In [7]:
# drop first column
sheet_df = sheet_df.drop(sheet_df.columns[0], axis=1)

In [8]:
sheet_df.head()

,Name,Tik Tok-Profil,Partei,aktiv,Anmerkungen,In fanpage karma
0,AfD,https://www.tiktok.com/@afd.offiziell,AfD,NaN,"Aktiv seit 10.05.2021, gelöscht 5/2022 von Tik...",NaN
1,AfD NRW,https://www.tiktok.com/@afd.nrw,AfD,x,NaN,NaN
2,AfD Hessen,https://www.tiktok.com/@afdhessen,AfD,NaN,Nicht mehr auffindbar,NaN
3,AfD Bayern,https://www.tiktok.com/@afd.bayern,AfD,NaN,Alter Account @afd_bayern nicht megr auffindbar,NaN
4,AfD Brandenburg,https://www.tiktok.com/@afd_brandenburg,AfD,x,NaN,NaN


In [9]:
# filter by "Partei" == "AfD"
afd_df = sheet_df[sheet_df['Partei'] == 'AfD']

In [12]:
len(afd_df)

174

In [18]:
# get Tiktok user name by "Tik Tok-Profil" column - currently a url like "https://www.tiktok.com/@afd.offiziell" from which we extract the user name "afd.offiziell"
afd_df['user_name'] = afd_df['Tik Tok-Profil'].str.extract(r'@(.*)')

# remove trailing whitespace
afd_df['user_name'] = afd_df['user_name'].str.strip()

/var/folders/ng/bpb66jzj5d59g10n121_hj6h0000gn/T/ipykernel_32162/1305649286.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  afd_df['user_name'] = afd_df['Tik Tok-Profil'].str.extract(r'@(.*)')
/var/folders/ng/bpb66jzj5d59g10n121_hj6h0000gn/T/ipykernel_32162/1305649286.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  afd_df['user_name'] = afd_df['user_name'].str.strip()


In [20]:
# drop all but user_name and Name columns
afd_df = afd_df[['user_name', 'Name']]

In [22]:
# save to csv
afd_df.to_csv('../../data/afd_accounts_tiktok.csv', index=False)

In [50]:
afd_df.head()

,user_name,Name
0,afd.offiziell,AfD
1,afd.nrw,AfD NRW
2,afdhessen,AfD Hessen
3,afd.bayern,AfD Bayern
4,afd_brandenburg,AfD Brandenburg


## Add AfD users to db

In [147]:
ms_tokens =["1-XrQyx6olgZKarwDXJ6I6bntG6PB-XY_diAn2nGOyq999caJb_yAezhU0FqpNYUpwyz89J5ydf7F59KnYoT9r7pHZTA6xsrnnZKgXwEG7o=","FH7W9nFjLoz17fqkqP_r01UTd-HusQvh-bERIzbg8yVi3Y01Aw7x2UPW-MGUZuhmo7cdsm1YpNnr1lJSBCFP-SQ70mj1-vVi-MZkyWkWFbA=", "d9k9wzoqvU6VocF-BCimm88shwMruwHNhtKfFq5W13n5EdCy3TpOpP2Ui7vRd7Hrr1144ivsBgJP7xvvNRAG21QGFpL81N8QOMxNHATTzNE=","rObrgyunEG5MX-wbESu_yhZz4Rr71w6c-W1ICQTVFbbNfTDwR7SnfYnTIC5OIvPnQpKX2UOtlnfDJLqh0f8qnxxqaD7W2BsaCeurTMT9F9k=","K6ycnFewW5yhPmBCFOe1PwyDvkXz_faFA6BMw8MmyxdRMxR2QpwiAZsNdHCps1nvHPJPn5R-PuJtlKImgAo08FYbRYLYBK4fL5YLnTtSd2I=", "rX-PrFF0AAPmO2xGm05ms_Kt_kozEvHd7pMM-yqZGhpdTxoF3B_RpNJItWLK6FlGSqLLtzLz1kcw7PmQ6WFNUZeUH-QzV7qHZjZk5E7I3k8cMghLvkAMMkA-0C_W", "eM76KGC4eqJ4E6QusDDIfj7msA293A6XAiArwRJFQfwUestV3quusqoiPiBWIOvY3S53zyhn0bV32m58wSyzTo2mMmZtHN7a3beZu0p7ock=", "SffxLpF-XLFk17jGiJoau1o_0NeAWzGBkAL47SeCqqHgs0uqHMvzQj9yYRhwtLzf65fY5BGuW3L9EXabrU7yOp_Xph6TchLMjKmWNr_x4Cg=", "Is5WghedY7a3L_-cBailDO3WmLXd94uz8MzHQzCwDj157LMDStwTRitRtP0C4bN05F_mJFwG47B0iNUBsOGyuDf5wqesvShJVMY67xGnKy0=", "u_h_df4QyHLarKVsk89WfQJ1te73jhZXwIAncCSPKyvZPqDxNUXRE25UwL3wNPJ3RbeLv_VU_bN0OtgfyVLgDbFtNOGQE642FaqR2U8VCj4=", "wW8HNaeovKQTiklnj_u5eGmyEQHlWFDa85rRMrrfkR4TZ3m99MBEPQ_6cY8ZN_CR8YTAXGGlI-igtsjei9U_x8nMdN3b6wonQ9YdaUg8qHM="] #os.environ.get("ms_token", None)

In [148]:
scrapper = Scrapper(ms_token=ms_tokens[random.randint(0, len(ms_tokens)-1)])

In [59]:
# get list of afd usernames:
afd_usernames = afd_df['user_name'].tolist()


In [ ]:
stats = user_info["userInfo"]["stats"]
user_id = user_info["userInfo"]["user"]["id"]
user_nickname = user_info["userInfo"]["user"]["nickname"]
user_name_unique = user_info["userInfo"]["user"]["uniqueId"]
verified = user_info["userInfo"]["user"]["verified"]

In [106]:
user_infos = asyncio.run(scrapper.get_user_info_by_username(count=1, user_names=afd_usernames[150:]))

Searching for pascalschleich915
Searching for richardgraupner
Searching for roland_magerl
Searching for udosteinmdl
Searching for _dieter_arnold_afd_
Searching for renedierkesafd
Searching for daniel_halemba_afd
Searching for andreasjurca.afd
Searching for benjamindnolte
Searching for ramona_storm
'user'
Searching for olgapetersenafd
Searching for alexander_wiesner
Searching for andre_wendt_afd
Searching for volkerdringenberg
Searching for frankpeschel_afd
Searching for rene_hein
Searching for steffenkubitzki
Searching for volker.nothing.md
Searching for felixhwteichner
Searching for hallofraukaiser
Searching for janeschulz.deu
'user'
Searching for dr.robbyschlund
Searching for jensahrends711
Searching for erichheidkamp


## add user to db

In [52]:
driver = os.environ["DB_DRIVER"]
server = os.environ["DB_SERVER"]
database = os.environ["DB_DATABASE"]
username = os.environ["DB_USERNAME"]
password = os.environ["DB_PASSWORD"]


connection_str = (
f"Driver={driver};Server={server},1433;Database={database};"
f"Uid={username};Pwd={password};Encrypt=yes;"
"TrustServerCertificate=no;Connection Timeout=30;"
)

In [107]:
# print number of nones in user_infos
print(len([i for i in user_infos if i is None]))

2


In [108]:
# add user info to database
scrapper.add_users_to_db_from_list_of_user_info(users=user_infos, connection_str=connection_str)

Number of users to be added to database 22
Number of unique user ids to be added to database 22
Number of unique user ids in database 2642
Number of unique user ids added to database 0
Number of new users to be added to database 0
Users added to database successfully


In [ ]:
# sql query to get all videos of users in afd_df
query = """
        SELECT * FROM dbo.Videos
        WHERE user_id = ?
        """

data = (int(user_id),)


In [ ]:
# sql query for retrieving the user_id by unique_name_id for a list of unique_name_ids
user_ids = []

for user_name_unique in afd_usernames:
        query = """
                SELECT id FROM dbo.Users
                WHERE unique_name_id = ?
                """
        
        data = (str(user_name_unique),)
        
        with pyodbc.connect(connection_str) as conn:
                with conn.cursor() as cursor:
                cursor.execute(query, data)
                user_id = cursor.fetchone()
        
        user_ids.append(user_id)





In [110]:
# filter nones from afd_usernames
afd_usernames_filtered = [i for i in afd_usernames if i is not None]

174

In [134]:
videos_by_user =[]

In [173]:
afd_usernames_filtered_subset = afd_usernames_filtered[:30]

In [177]:
# chunk the unique_name_ids list and loop through them
import random

def get_new_ms_token(scrapper, ms_tokens):
    # random select of ms_token in ms_token list without removing the token from the list
    while True:
        random_idx = random.randint(0, len(ms_token)-1)
        new_ms_token = ms_token[random_idx]
        if new_ms_token != scrapper.ms_token:
            print(f"New ms_token: {new_ms_token}")
            return new_ms_token

chunk_size = 10

for i in range(0, len(afd_usernames_filtered_subset), chunk_size):
    print(i)
    print(scrapper.ms_token)
    try:
        videos_by_user = asyncio.run(scrapper.search_videos_by_users(count=29, users=afd_usernames_filtered_subset[i:i+chunk_size], videos=videos_by_user))
    except Exception as e:
        print(e)
        new_ms_token = get_new_ms_token(scrapper, ms_tokens)
        scrapper.update_ms_token(ms_token=new_ms_token)
        #time.sleep(2)
        continue
    new_ms_token = get_new_ms_token(scrapper, ms_tokens)
    scrapper.update_ms_token(ms_token=new_ms_token)
    print(len(videos_by_user))

0
SffxLpF-XLFk17jGiJoau1o_0NeAWzGBkAL47SeCqqHgs0uqHMvzQj9yYRhwtLzf65fY5BGuW3L9EXabrU7yOp_Xph6TchLMjKmWNr_x4Cg=
Searching for afd.offiziell
User afd.offiziell not found
Searching for afd.nrw
{'extra': {'fatal_item_ids': [], 'logid': '202405161204100E092A96E6F9F822BBE1', 'now': 1715861051000}, 'log_pb': {'impr_id': '202405161204100E092A96E6F9F822BBE1'}, 'shareMeta': {'desc': '@afd.nrw 47.8k Followers, 7 Following, 422.2k Likes - Watch awesome short videos created by AfD.NRW', 'title': 'AfD.NRW on TikTok'}, 'statusCode': 0, 'status_code': 0, 'status_msg': '', 'userInfo': {'stats': {'diggCount': 0, 'followerCount': 47800, 'followingCount': 7, 'friendCount': 3, 'heart': 422200, 'heartCount': 422200, 'videoCount': 62}, 'user': {'avatarLarger': 'https://p16-sign-va.tiktokcdn.com/tos-maliva-avt-0068/64d39670ed9fd285cbb675f1e57853d3~c5_1080x1080.jpeg?lk3s=a5d48078&nonce=58178&refresh_token=e93c11063ff3cb50ee3028e9b37dfaa6&x-expires=1716033600&x-signature=F8BE%2BgeDkjmHEBcSw0s%2FP9x0sVU%3D&shp=a

In [178]:
len(videos_by_user)

6249

In [179]:

# Process each video and collect the processed data
processed_videos_by_user = [scrapper.process_video(video) for video in scrapper.filter_unique_videos(videos_by_user)]

print(len(processed_videos_by_user))


3428


In [171]:
scrapper.add_sounds_to_db(processed_videos_by_user, connection_str=connection_str)

Number of new sounds to be added to database 0
Sounds added to database successfully


In [172]:
scrapper.add_videos_to_db(processed_videos_by_user, connection_str=connection_str)

Number of new videos to be added to database 0
Videos added to database successfully


In [186]:
# query database and filter for videos which are NULL in has_transcript column
with pyodbc.connect(connection_str) as cnxn:
    cursor = cnxn.cursor()
    query = """
    SELECT * FROM dbo.Videos
    WHERE has_transcript = 0
    """
    cursor.execute(query)
    rows = cursor.fetchall()
    print(len(rows))

3833
